In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
url = '/home/nkiru/DS-Unit-2-Applied-Modeling/data/lending-club/lending-club-subset.csv'
#/home/nkiru/DS-Unit-2-Applied-Modeling/data/lending-club

In [3]:
df = pd.read_csv(url)
print(df.shape)
df.head()

(128334, 150)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,109889419.0,NaN,20000.0,20000.0,20000.0,36 months,9.44%,640.10,B,B1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68579794.0,NaN,21600.0,21600.0,21600.0,36 months,9.80%,694.95,B,B3,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,10105952.0,NaN,25000.0,25000.0,24950.0,36 months,17.57%,898.43,D,D2,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,57713137.0,NaN,5000.0,5000.0,5000.0,36 months,6.89%,154.14,A,A3,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,11966074.0,NaN,11850.0,11850.0,11850.0,36 months,14.16%,405.93,C,C2,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Define the wrangle function
def wrangle(X):
    X = X.copy()
    X['issue_d'] = pd.to_datetime(X['issue_d'], infer_datetime_format=True)
    # Just use 36 month loans
    X = X[X.term==' 36 months']
 

    # Index & sort by issue date
    X = X.set_index('issue_d').sort_index()

    # Clean data, engineer feature, & select subset of features
    X = X.rename(columns=
                 {'annual_inc': 'Annual Income',
                  'fico_range_high': 'Credit Score',
                  'funded_amnt': 'Loan Amount',
                  'title': 'Loan Purpose'})
    X['Interest Rate'] = X['int_rate'].str.strip('%').astype(float)
    X['Monthly Debts'] = X['Annual Income'] / 12 * X['dti'] / 100
    
    columns = ['Annual Income',
               'Credit Score',
               'Loan Amount',
               'Loan Purpose',
               'Monthly Debts',
               'Interest Rate']
    X = X[columns]
    #X['Loan Purpose'] = X['Loan Purpose'].apply(lambda x: x.lower())
    X.columns = [col.replace(' ', '_') for col in X]

    
  
    X = X.dropna()
    # return the wrangled dataframe
    return X

In [5]:
df = wrangle(df)

In [6]:
target = 'Interest_Rate'
features = ['Annual_Income', 'Credit_Score', 'Loan_Amount', 'Loan_Purpose','Monthly_Debts']
X = df[features]
y = df[target]

In [7]:
#import train_test_split
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)
#print('X_train shape:', X_train.shape, 'X_test shape:', X_test.shape, 'y_train shape:', y_train.shape, 'y_test:', y_test.shape)

In [8]:
test = df[-10000:]
val = df[-20000:-10000]
train = df[:-20000]

In [9]:
X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = test[features]
y_test = test[target]

In [10]:
X_train.head()

,Annual_Income,Credit_Score,Loan_Amount,Loan_Purpose,Monthly_Debts
issue_d,,,,,
2007-07-01,150000.0,779.0,5000.0,New kitchen,0.000000
2007-07-01,200000.0,679.0,3000.0,Roofdeck for my new NYC apartment,0.000000
2007-07-01,95000.0,774.0,5000.0,Mortgage Pre-Payment,303.208333
2007-07-01,20000.0,684.0,3500.0,Renting Apartment: Need security deposit,25.000000
2007-08-01,61800.0,749.0,4000.0,Apartment,178.190000


In [11]:
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from category_encoders import OrdinalEncoder
from sklearn.model_selection import cross_val_score

print('setup complete')

setup complete


In [12]:
xgb_model = make_pipeline(
    OrdinalEncoder(),
    StandardScaler(),
    SimpleImputer(strategy = 'mean'),
    XGBRegressor(
        learning_rate = 0.01,
        max_depth = None,
        n_estimators = 512,
        scale_pos_weight = 1,
        random_state = 42,
        n_jobs = -1)
)
xgb_model.fit(X_train, y_train);

In [13]:
def predict(annual_income, credit_score, loan_amount, loan_purpose, monthly_debts):
    df = pd.DataFrame(
        columns  = ['Annual_Income', 'Credit_Score', 'Loan_Amount', 'Loan_Purpose','Monthly_Debts'],
        data = [[annual_income, credit_score, loan_amount, loan_purpose, monthly_debts]]
    )

    y_pred = xgb_model.predict(df)[0]
    return f'The model predicted {y_pred:}% interest rate '

In [14]:
#ypred = xgb_model.predict(X_test)
#ypred

In [15]:
from joblib import dump
dump(xgb_model, 'xgb_model.joblib', compress=True)

['xgb_model.joblib']